# EIA average monthly retail price of electricity by state

Some Python to hit the Energy Information Administration API to collect historical monthly average retail electricity prices by state (from the [Electric Power Monthly report](https://www.eia.gov/electricity/monthly/), which AFAIK is not available at this granual level from the agency's bulk data), and also to nab Bureau of Labor Statistics inflation data (annual averages of the [CPI-All Urban Consumers](https://data.bls.gov/timeseries/CUUR0000SA0?output_view=pct_1mth)) to adjust the numbers.

To run this notebook:
- Get [pipenv](https://docs.pipenv.org) if you don't have it already
- Install the dependencies in the Pipfile: `pipenv install`
- [Get your EIA API key here](https://www.eia.gov/developer/) and save it as an environmental variable called `EIA_API_KEY`
- [Get your BLS API key here](https://data.bls.gov/registrationEngine/) and save it as an environmental variable called `BLS_API_KEY`

### Import libraries

In [1]:
import os
import time
import csv
import json
from itertools import groupby

import requests
import us
import pandas as pd

### Define some key variables and helper functions

In [25]:
EIA_API_KEY = os.environ.get('EIA_API_KEY')
BLS_API_KEY = os.environ.get('BLS_API_KEY')

EIA_ENDPOINT_TEMPLATE = 'http://api.eia.gov/series/?api_key={key}&series_id=ELEC.PRICE.{state}-ALL.M'
BLS_ENDPOINT = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

EIA_CSV_FILE = 'avg-retail-price.csv'
BLS_CSV_FILE = 'cpi-annual-averages.csv'

CPI_SERIES = 'CUUR0000SA0'


def mean(numbers):
    '''Given a list of numbers, return the arithmatic mean'''
    return round(float(sum(numbers)) / max(len(list(numbers)), 1), 3)


def calculate_inflation(price=0, base_year_index=100.0, current_year_index=100.0):
    '''Given a price, a base year index and the current year index, return the adjusted value
       See: https://www.bls.gov/cpi/factsheets/cpi-math-calculations.pdf#page=2
       "constant dollars"
    '''
    return round((float(base_year_index)/float(current_year_index))*price, 2)

### Get EIA data, write to file

In [3]:
with open(EIA_CSV_FILE, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['state', 'month', 'year', 'price'])
    writer.writeheader()

    for state in us.states.STATES:
        d = {'state': state.abbr}
        endpoint = EIA_ENDPOINT_TEMPLATE.format(key=EIA_API_KEY, state=state.abbr)
        r = requests.get(endpoint)

        data = r.json()['series'][0]['data']
        for m in data:
            d['year'] = m[0][:4]
            d['month'] = m[0][-2:]
            d['price'] = m[1]
            writer.writerow(d)

        time.sleep(2)

### Get BLS data, write to file

In [5]:
with open(BLS_CSV_FILE, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['year', 'avg'])
    writer.writeheader()

    headers = {'Content-type': 'application/json'}
    
    """
    We have to break it up into two API calls to handle the limitations on
    the amount of data returned    
    """
    years = (('2000', '2008'), ('2009', '2017'))

    for y in years:
        data = json.dumps({'seriesid': [CPI_SERIES],
                           'startyear': y[0],
                           'endyear': y[1]})

        r = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/',
                          data=data, headers=headers, auth=(BLS_API_KEY, ''))
        r.raise_for_status()

        data = r.json()['Results']['series'][0]['data']
        grouped_data = groupby(data, lambda x: x['year'])

        for year, months in grouped_data:
            avg = mean([float(x['value']) for x in months])
            writer.writerow({'year': year, 'avg': avg})

        time.sleep(2)

In [10]:
# load EIA data into a dataframe
df = pd.read_csv(EIA_CSV_FILE)

# kill 2018 data
df = eia_df[eia_df['year'] != 2018]

In [11]:
eia_df.head()

,state,month,year,price
1,AL,12,2017,9.53
2,AL,11,2017,9.67
3,AL,10,2017,9.82
4,AL,9,2017,10.33
5,AL,8,2017,10.25


In [29]:
# open BLS data file and turn it into a dict
with open(BLS_CSV_FILE, 'r') as b:
    cpi = [x.split(',') for x in b.read().splitlines()]
    cpi.pop(0)

cd = dict(cpi)
cpi_dict = {int(x): float(cd[x]) for x in cd}

In [26]:
df['adjusted_price'] = df.apply(lambda x: calculate_inflation(price=x['price'],
                                                              base_year_index=cpi_dict[2017],
                                                              current_year_index=cpi_dict[x['year']]), axis=1)

In [30]:
df.tail()

,state,month,year,price,adjusted_price
10450,WY,5,2001,4.40,6.09
10451,WY,4,2001,4.26,5.90
10452,WY,3,2001,4.34,6.01
10453,WY,2,2001,4.36,6.04
10454,WY,1,2001,4.31,5.97


In [ ]:
# TK: plot by state